In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sunau

STRING_MAP = ['A\tmajor',
             'A#\tmajor',
             'B\tmajor',
             'C\tmajor',
             'C#\tmajor',
             'D\tmajor',
             'D#\tmajor',
             'E\tmajor',
             'F\tmajor',
             'F#\tmajor',
             'G\tmajor',
             'G#\tmajor',
             'A\tminor',
             'A#\tminor',
             'B\tminor',
             'C\tminor',
             'C#\tminor',
             'D\tminor',
             'D#\tminor',
             'E\tminor',
             'F\tminor',
             'F#\tminor',
             'G\tminor',
             'G#\tminor']


In [31]:
def get_vector_from_key(key):
    vector = np.zeros(24)
    if key < 12: #major
        vector[key] = 1
        vector[(key + 7) % 12] = 0.5
        vector[(key + 9) % 12 + 12] = 0.3
        vector[key + 12] = 0.2
    else: # minor
        vector[key] = 1
        vector[(key + 7) % 12 + 12] = 0.5
        vector[(key + 3) % 12] = 0.3
        vector[key - 12] = 0.2
        
    vector /= 2
    
    return vector

In [38]:
def get_string_from_vector(vector):
    return STRING_MAP[np.argmax(vector)]

In [39]:
def generate_one_hot_guess(vector):
    return np.argmax(vector)

In [ ]:
def read_au_file(infile):
    read_file = sunau.open(infile, 'r')
    fs = read_file.getframerate()

In [34]:
vector = np.array([0, 0, 0, 1, 0])

In [35]:
vector

array([0, 0, 0, 1, 0])

In [37]:
np.argmax(vector)

3